### Linear Regression using TensorFlow

**Predicting the price of a car using a neural network mindset**

#### Let us start with mounting our drive to be able to use the dataset stored in our dataset.

Install the following: (Optional, Not needed for this analysis, used when connecting directly to google.colab)

1. pip install google

2. pip install --upgrade google-api-python-client

3. pip install google-colab

In [1]:
# import google
# from google.colab import drive
# # drive.mount('/content/drive')

#### Now, let us install the latest version of tensorflow.


In [2]:
# !pip install tensorflow

##### Check the version of the installation

In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


Now Let us import out data and get it ready for modelling.

In [4]:
import pandas as pd
import numpy as np

#### Read the required Dataset

In [5]:
#--Read the usedcars.csv and store it in the data frame, cars_data. You need to replace None with the appropriate syntax
cars_data = pd.read_csv('usedcars.csv')
cars_data.head()

,year,model,price,mileage,color,transmission
0,2011,SEL,21992,7413,Yellow,AUTO
1,2011,SEL,20995,10926,Gray,AUTO
2,2011,SEL,19995,7351,Silver,AUTO
3,2011,SEL,17809,11613,Gray,AUTO
4,2012,SE,17500,8367,White,AUTO


#### Checking for missing value

In [6]:
cars_data.isna().sum().sum()

0

#### Let's drop duplicates if any

In [7]:
#just some basic preprocessing
cars_data.drop_duplicates(inplace=True)

#### Converting categorical variables to dummies

use the pd.get_dummies() to create dummy of all categorical variable. Note we won't be using drop_first=True for the following analysis here. Replace "None" in the code below with the correct syntax.

In [8]:

#creating dummy variables for the categorical features
cars_data = pd.get_dummies(cars_data)

cars_data = cars_data.astype('float32') # we will need to convert the dataset to float in order to be able to convert it into tensors later.
cars_data.head()


,year,price,mileage,model_SE,model_SEL,model_SES,color_Black,color_Blue,color_Gold,color_Gray,color_Green,color_Red,color_Silver,color_White,color_Yellow,transmission_AUTO,transmission_MANUAL
0,2011.0,21992.0,7413.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2011.0,20995.0,10926.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2011.0,19995.0,7351.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2011.0,17809.0,11613.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2012.0,17500.0,8367.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [9]:
cars_data.shape

(150, 17)

In [10]:
#explorinfg column names
cars_data.columns

Index(['year', 'price', 'mileage', 'model_SE', 'model_SEL', 'model_SES',
       'color_Black', 'color_Blue', 'color_Gold', 'color_Gray', 'color_Green',
       'color_Red', 'color_Silver', 'color_White', 'color_Yellow',
       'transmission_AUTO', 'transmission_MANUAL'],
      dtype='object')

**Single Neuron Neural Network for Linear Regression**:

**Also known as - Single neuron == linear regression without applying activation(perceptron) **

<img src="TF-Output3.png" style="width:800px;height:500px;">

**Reminder**: 

The general methodology to build a Neural Network is to:
    1. Define the neural network structure ( # of input units,  # of hidden units, etc). 
    2. Initialize the model's parameters
    3. Loop:
        - Implement forward propagation
        - Compute loss
        - Implement backward propagation to get the gradients
        - Update parameters (gradient descent)

#### Creating training and test data

Note the dependent variable is price, rest are independent variable. Replace "None" below with the required column names.

In [11]:
#getting the features and labels and finally splitting the test and train data.


from sklearn.model_selection import train_test_split

X = cars_data[['year',  'mileage', 'model_SE', 'model_SEL', 'model_SES',
       'color_Black', 'color_Blue', 'color_Gold', 'color_Gray', 'color_Green',
       'color_Red', 'color_Silver', 'color_White', 'color_Yellow',
       'transmission_AUTO', 'transmission_MANUAL']]
Y = cars_data['price']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [12]:
X_train.shape

(120, 16)

In [13]:
#let us scale the data as features are on different scales which might be a problem while modelling
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[0.75      , 0.27928138, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.58332825, 0.5233474 , 0.        , ..., 1.        , 0.        ,
        1.        ],
       [0.75      , 0.21721278, 1.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.83332825, 0.20944397, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.33332825, 0.6878496 , 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.83332825, 0.2253431 , 1.        , ..., 0.        , 1.        ,
        0.        ]], dtype=float32)

#### Inorder to use Tensor Flow for this analysis, we need convert the datasets into tensors

In [14]:
# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need 
#to be transformed into numpy arrays.

X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)


#### Defining the neural network structure
Exercise: Define three variables:

- n_x: the size of the input layer (i.e. number of independent variables)
- n_h: the size of the hidden layer (set this to 1, number of neurons) 
- n_y: the size of the output layer (i.e. number of output variable)
Hint: Use shapes of X and Y to find n_x and n_y. Also, hard code the hidden layer size to be 1.

In [15]:
def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    n_x = X.shape[0] # size of input layer
    n_h = 1
    n_y = 1 # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [16]:
(n_x, n_h, n_y) = layer_sizes(X_train, y_train)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

The size of the input layer is: n_x = 120
The size of the hidden layer is: n_h = 1
The size of the output layer is: n_y = 1


In [17]:
display("X Train:",X_train.shape)
display("X Test:",y_test.shape)
display("Y Train:",X_train.shape)
display("Y Test:",y_test.shape)

'X Train:'

TensorShape([120, 16])

'X Test:'

TensorShape([30])

'Y Train:'

TensorShape([120, 16])

'Y Test:'

TensorShape([30])

#### We need to declare Weights and bias of the data

In [18]:
input_dim = X_train.shape[1]
output_dim = 1
learning_rate = 0.01

# Let us initialize the weights and bias variables. 
weights = tf.Variable(tf.zeros(shape=(input_dim, output_dim), dtype= tf.float32))
display(weights)

bias = tf.Variable(tf.ones(shape=(output_dim,), dtype= tf.float32))
display(bias)

<tf.Variable 'Variable:0' shape=(16, 1) dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>

**Note:**

1. Since we have 16 input/independent variables, we need to have 16 weights

2. We are utilizing one neuron here, hence we need one bias term

**Note: All variables are stored as matrices, the input variable is a tensor matrix of size 120 x 16, the weight variable is a matrix of shape 16 x 1 and bias is set to 0. We need to perform a matrix multiplication to obtain the equation for every output.**

If you are familiar with how matrix multiplication works, you will realise that the output will be a matrix of size 120 x 1

<img src="TF-Output4.jpg" style="width:800px;height:500px;">


Functions used below:

1. matmul: Multiplies matrix a by matrix b, producing a * b.

2. reduce_mean: Computes the mean of elements across dimensions of a tensor

3. GradientTape(): Record operations for automatic differentiation.


**Update the "None" in predict() function using the concepts from above**

In [19]:
def predict(features):
  return tf.matmul(features, weights) + bias # note that the matmul is matrix multiplication and is needed for calculating predictions

def compute_loss(y_true, predictions):
  return tf.reduce_mean(tf.square(y_true - predictions)) # mean square error

**Once the output is calculated, we need to recalculate the weights using gradient descent**

**Question**: Implement the update rule. Use gradient descent. You have to use derivative of weights and bias 
in order to update weights and bias.

**General gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ where $\alpha$ is the learning rate and $\theta$ represents a parameter.

**Illustration**: The gradient descent algorithm with a good learning rate (converging) and a bad learning rate (diverging). Images courtesy of Adam Harley.

<img src="sgd.gif" style="width:400;height:400;"> <img src="sgd_bad.gif" style="width:400;height:400;">

In [20]:
# Let us now define a function to train the model. We will call the other functions in function definition.
def train(x, y):
    
    #-------------Recalibrating the neural network
    with tf.GradientTape() as tape:
        
        #----------------------Forward Propagation-----------------------------------

        predictions = predict(x)
        loss = compute_loss(y, predictions)

        #--------------------Backward Propagation------------------------------------

        dloss_dw, dloss_db = tape.gradient(loss, [weights, bias]) #note that we can pass lists as well here.

        #--------------------Reassigning Weights based on learning rate and derivatives

    weights.assign_sub(learning_rate * dloss_dw)
    bias.assign_sub(learning_rate * dloss_db)

    return loss 

#### Let us now, call the train function with 500 epochs

In [23]:

for epoch in range(500):
  loss = train(X_train, y_train)
#   print('Epoch %d: Loss = %.4f' % (epoch, float(loss)))


print('Final Weights after 500 epochs:')
print('###############################################################################')
print(weights)

print('Final Bias after 500 epochs:')
print('###############################################################################')
print(bias)

Final Weights after 500 epochs:
###############################################################################
<tf.Variable 'Variable:0' shape=(16, 1) dtype=float32, numpy=
array([[2697.8462 ],
       [1773.5265 ],
       [1890.4213 ],
       [1579.4885 ],
       [1817.8314 ],
       [ 654.1514 ],
       [ 674.8424 ],
       [  93.19248],
       [ 721.20557],
       [ 674.38544],
       [ 741.1054 ],
       [ 660.46313],
       [ 544.62646],
       [ 523.7726 ],
       [2692.5276 ],
       [2595.219  ]], dtype=float32)>
Final Bias after 500 epochs:
###############################################################################
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([5288.746], dtype=float32)>


#### Let us now test our model on the test data and predict on the test data.

In [22]:
test_predictions = tf.matmul(X_test, weights) + bias
print(compute_loss(y_test, test_predictions))

tf.Tensor(6689722.0, shape=(), dtype=float32)
